# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core.workspace import Workspace

In [2]:
ws = Workspace.from_config()

In [3]:
from azureml.core import Environment

sklearn_env = Environment.get(workspace=ws, name='AzureML-Tutorial')

In [4]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.60.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [6]:

print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

mlcloud
mlcloud
eastus2
d990bb6c-7849-4109-9dd7-6cafa051c8ae


In [7]:
# choose a name for experiment
experiment_name = 'CapstoneProj'

experiment=Experiment(ws, experiment_name)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [9]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "project2"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_v3',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.


Get dataset

In [10]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "loandataset"
description_text = "Loan Dataset for Capstone Project"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,person_age,person_income,person_emp_exp,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,loan_status
count,45000.000000,4.500000e+04,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000
mean,27.764178,8.031905e+04,5.410333,9583.157556,11.006606,0.139725,5.867489,632.608756,0.222222
std,6.045108,8.042250e+04,6.063532,6314.886691,2.978808,0.087212,3.879702,50.435865,0.415744
min,20.000000,8.000000e+03,0.000000,500.000000,5.420000,0.000000,2.000000,390.000000,0.000000
25%,24.000000,4.720400e+04,1.000000,5000.000000,8.590000,0.070000,3.000000,601.000000,0.000000
50%,26.000000,6.704800e+04,4.000000,8000.000000,11.010000,0.120000,4.000000,640.000000,0.000000
75%,30.000000,9.578925e+04,8.000000,12237.250000,12.990000,0.190000,8.000000,670.000000,0.000000
max,144.000000,7.200766e+06,125.000000,35000.000000,20.000000,0.660000,30.000000,850.000000,1.000000


In [11]:
project_folder = './pipeline-project'

In [13]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="loan_status",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [14]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
CapstoneProj,AutoML_7e5b9b85-56f9-4647-b10a-dc7828aa1612,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [17]:
# Get best run and model
best_run, fitted_model = remote_run.get_output()

# Print the best run
print(best_run)

# Get all metrics of the best run
best_run_metrics = best_run.get_metrics()

# Print all metrics of the best run
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Run(Experiment: CapstoneProj,
Id: AutoML_7e5b9b85-56f9-4647-b10a-dc7828aa1612_37,
Type: azureml.scriptrun,
Status: Completed)
AUC_weighted 0.9819242857142858
norm_macro_recall 0.7847142857142857
AUC_micro 0.988108
precision_score_macro 0.9292378742058747
average_precision_score_weighted 0.9845126073021082
average_precision_score_macro 0.9717243901533279
accuracy 0.9397777777777778
recall_score_micro 0.9397777777777778
f1_score_weighted 0.9384274473571006
log_loss 0.16003347409255533
f1_score_micro 0.9397777777777778
balanced_accuracy 0.8923571428571428
AUC_macro 0.9819242857142858
average_precision_score_micro 0.9885433636737777
precision_score_weighted 0.9388898005537714
f1_score_macro 0.9090724382119444
matthews_correlation 0.8207668266432087
precision_score_micro 0.9397777777777778
recall_score_weighted 0.9397777777777778
recall_score_macro 0.8923571428571428
weighted_accuracy 0.9648301886792452
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_7e5b9b85-56f9-4647-b10a-dc78

In [24]:
# Register the model to deploy
automl_model = best_run.register_model(model_name = "automl_capstone_bestmodel", model_path = "outputs/model.pkl")
print(automl_model)

Model(workspace=Workspace.create(name='mlcloud', subscription_id='d990bb6c-7849-4109-9dd7-6cafa051c8ae', resource_group='mlcloud'), name=automl_capstone_bestmodel, id=automl_capstone_bestmodel:2, version=2, tags={}, properties={})


In [20]:
from azureml.core import Model

In [21]:
# List registered models to verify if model has been saved
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

automl_capstone_bestmodel version: 1


udacityprojec221 version: 1


project218 version: 1


bank_marketing_best_model version: 1


bikeautoml9 version: 1


best-logistic-regression-model version: 2
	 Training context : HyperDrive


best-automl-model version: 1
	 Training context : AutoML


best-logistic-regression-model version: 1
	 Training context : HyperDrive


fashionhaven-sales-predictor-gbr version: 5
	 flavors.python_function : {
  "model_path": "model.pkl",
  "predict_fn": "predict",
  "loader_module": "mlflow.sklearn",
  "python_version": "3.8.16",
  "env": {
    "conda": "conda.yaml",
    "virtualenv": "python_env.yaml"
  }
}
	 flavors.sklearn : {
  "pickled_model": "model.pkl",
  "sklearn_version": "1.0.2",
  "serialization_format": "cloudpickle",
  "code": null
}
	 flavors : python_function,sklearn
	 azureml.artifactPrefix : ExperimentRun/dcid.9252b192-4db5-44cc-882c-976af4ee5dd9/fashionhaven-sales-predictor-gbr
	 model_json : {"run_id": "9252b192-4db5-44cc-882c-976af4ee5d

In [25]:
automl_model

Model(workspace=Workspace.create(name='mlcloud', subscription_id='d990bb6c-7849-4109-9dd7-6cafa051c8ae', resource_group='mlcloud'), name=automl_capstone_bestmodel, id=automl_capstone_bestmodel:2, version=2, tags={}, properties={})

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [22]:
# Download scoring file 
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'scoreScript.py')

# Download environment file
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'envFile.yml')


In [26]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script='scoreScript.py',
                                    environment=best_run.get_environment())

# deployauth_header = interactive_auth.get_authentication_header()
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "automlservice", [automl_model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

print(service.scoring_uri)

print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-05-10 22:33:59+00:00 Creating Container Registry if not exists.
2025-05-10 22:33:59+00:00 Generating deployment configuration.
2025-05-10 22:34:02+00:00 Submitting deployment to compute.
2025-05-10 22:34:06+00:00 Checking the status of deployment automlservice..
2025-05-10 22:37:43+00:00 Checking the status of inference endpoint automlservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://0cf9ff93-42e7-4444-b612-84121d2240c7.eastus2.azurecontainer.io/score
http://0cf9ff93-42e7-4444-b612-84121d2240c7.eastus2.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [30]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

In [27]:
df.head()

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.00,female,Master,71948.00,0,RENT,35000.00,PERSONAL,16.02,0.49,3.00,561,False,1
1,21.00,female,High School,12282.00,0,OWN,1000.00,EDUCATION,11.14,0.08,2.00,504,True,0
2,25.00,female,High School,12438.00,3,MORTGAGE,5500.00,MEDICAL,12.87,0.44,3.00,635,False,1
3,23.00,female,Bachelor,79753.00,0,RENT,35000.00,MEDICAL,15.23,0.44,2.00,675,False,1
4,24.00,male,Master,66135.00,1,RENT,35000.00,MEDICAL,14.27,0.53,4.00,586,False,1


In [28]:
import json

#Import test data
test_data = df.sample(10) # data is the pandas dataframe of the original data
label_df = test_data.pop('loan_status')

json_sample = json.dumps({'data': test_data.to_dict(orient='records')})

print(json_sample)

{"data": [{"person_age": 30.0, "person_gender": "female", "person_education": "Master", "person_income": 29948.0, "person_emp_exp": 5, "person_home_ownership": "RENT", "loan_amnt": 3000.0, "loan_intent": "VENTURE", "loan_int_rate": 9.77, "loan_percent_income": 0.1, "cb_person_cred_hist_length": 7.0, "credit_score": 613, "previous_loan_defaults_on_file": true}, {"person_age": 24.0, "person_gender": "female", "person_education": "High School", "person_income": 78886.0, "person_emp_exp": 2, "person_home_ownership": "RENT", "loan_amnt": 8000.0, "loan_intent": "EDUCATION", "loan_int_rate": 12.69, "loan_percent_income": 0.1, "cb_person_cred_hist_length": 3.0, "credit_score": 644, "previous_loan_defaults_on_file": true}, {"person_age": 31.0, "person_gender": "female", "person_education": "Associate", "person_income": 48241.0, "person_emp_exp": 8, "person_home_ownership": "RENT", "loan_amnt": 2500.0, "loan_intent": "MEDICAL", "loan_int_rate": 15.27, "loan_percent_income": 0.05, "cb_person_cred

In [36]:
type(json_sample)

str

In [37]:
import urllib.request
import json

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
# data = json_sample
data = json.loads(json_sample)
body = str.encode(json.dumps(data))

url = 'http://0cf9ff93-42e7-4444-b612-84121d2240c7.eastus2.azurecontainer.io/score'


headers = {'Content-Type':'application/json', 'Accept': 'application/json'}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'"{\\"result\\": [0, 0, 0, 0, 0, 1, 0, 0, 0, 1]}"'


TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
